In [37]:
from sklearn.preprocessing import Normalizer,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,PoissonRegressor
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import  xgboost  as xg
import pandas as pd 
import numpy as np
import pickle
import os
import json

In [39]:
#Folder of recored models
direc= "ml_records"
#Loading datamart
data_dir="C:\\Users\\Arda\\Downloads\\football_master\\Documantation\\Datas\\"
data=pd.read_csv("{}datamart.csv".format(data_dir))

In [40]:
data.columns

Index(['fixture.id', 'fixture.timezone', 'fixture.venue.city',
       'fixture.status.long', 'fixture.status.short', 'fixture.status.elapsed',
       'league.id', 'league.name', 'league.country', 'league.season',
       ...
       'Rank_Name_last_2_3_Away', 'Rank_Name_title_Away',
       'Rise_Need_1_point_needed_Away', 'Rise_Need_3_point_for_catch_Away',
       'Rise_Need_3_point_for_chase_Away', 'Rise_Need_top_Away',
       'Fall_Prevent_1_point_for_run_Away',
       'Fall_Prevent_3_point_for_run_Away', 'Fall_Prevent_bottom_Away',
       'Fall_Prevent_comfort_Away'],
      dtype='object', length=139)

In [41]:
#Selecting incoming matches
incoming_matches= data[data["fixture.status.short"]=="NS"].reset_index(drop=True)
incoming_matches

,fixture.id,fixture.timezone,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.season,...,Rank_Name_last_2_3_Away,Rank_Name_title_Away,Rise_Need_1_point_needed_Away,Rise_Need_3_point_for_catch_Away,Rise_Need_3_point_for_chase_Away,Rise_Need_top_Away,Fall_Prevent_1_point_for_run_Away,Fall_Prevent_3_point_for_run_Away,Fall_Prevent_bottom_Away,Fall_Prevent_comfort_Away
0,1035414,UTC,"Brentford, Middlesex",Not Started,NS,NaN,39,Premier League,England,2023,...,0,1,0,0,0,1,0,1,0,0
1,1035415,UTC,Burnley,Not Started,NS,NaN,39,Premier League,England,2023,...,0,0,0,1,0,0,0,1,0,0
2,1035416,UTC,Liverpool,Not Started,NS,NaN,39,Premier League,England,2023,...,0,0,0,0,1,0,1,0,0,0
3,1035417,UTC,London,Not Started,NS,NaN,39,Premier League,England,2023,...,0,0,0,1,0,0,0,0,0,1
4,1035418,UTC,"Luton, Bedfordshire",Not Started,NS,NaN,39,Premier League,England,2023,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,1146787,UTC,"Itu, São Paulo",Not Started,NS,NaN,475,Paulista - A1,Brazil,2024,...,0,0,0,1,0,0,0,1,0,0
230,1146788,UTC,"São Paulo, São Paulo",Not Started,NS,NaN,475,Paulista - A1,Brazil,2024,...,0,0,0,1,0,0,0,1,0,0
231,1146789,UTC,"Bragança Paulista, São Paulo",Not Started,NS,NaN,475,Paulista - A1,Brazil,2024,...,0,0,0,0,1,0,0,1,0,0
232,1146790,UTC,"São Bernardo do Campo, São Paulo",Not Started,NS,NaN,475,Paulista - A1,Brazil,2024,...,0,0,1,0,0,0,0,1,0,0


In [42]:
#Selecting data with feature columns
feature_data=incoming_matches.iloc[:,27:]
feature_data

,Foul_Home,Corner_Kicks_Home,Offsides_Home,Ball_Possession_Rate_Home,Yellow_Cards_Home,Red_Cards_Home,Total_Passes_Home,Accurate_Pass_Rate_Home,Total_Shot_Home,Shot_On_Goal_Rate_Home,...,Rank_Name_last_2_3_Away,Rank_Name_title_Away,Rise_Need_1_point_needed_Away,Rise_Need_3_point_for_catch_Away,Rise_Need_3_point_for_chase_Away,Rise_Need_top_Away,Fall_Prevent_1_point_for_run_Away,Fall_Prevent_3_point_for_run_Away,Fall_Prevent_bottom_Away,Fall_Prevent_comfort_Away
0,9.5,5.0,1.0,0.300,2.0,0.0,277.5,0.695,9.0,0.620,...,0,1,0,0,0,1,0,1,0,0
1,12.0,2.5,1.5,0.340,2.0,0.0,306.0,0.720,10.5,0.500,...,0,0,0,1,0,0,0,1,0,0
2,11.5,5.0,3.5,0.310,2.0,0.0,276.0,0.710,12.0,0.280,...,0,0,0,0,1,0,1,0,0,0
3,8.0,8.5,1.0,0.530,1.0,0.0,471.0,0.770,14.5,0.520,...,0,0,0,1,0,0,0,0,0,1
4,12.5,7.5,3.0,0.535,1.5,0.0,460.5,0.810,16.0,0.710,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,10.0,3.0,1.0,0.500,3.0,0.0,382.0,0.775,7.0,0.500,...,0,0,0,1,0,0,0,1,0,0
230,15.0,5.5,1.0,0.490,3.0,0.0,404.5,0.835,12.0,0.340,...,0,0,0,1,0,0,0,1,0,0
231,13.5,4.5,0.0,0.570,1.5,0.0,391.0,0.765,11.5,0.285,...,0,0,0,0,1,0,0,1,0,0
232,11.0,2.0,1.0,0.480,2.0,0.0,399.5,0.850,9.5,0.500,...,0,0,1,0,0,0,0,1,0,0


In [43]:
#Selecting data with relevant to prediction
incoming_matches_v1=pd.concat([incoming_matches[["fixture.id"]].reset_index(drop=True),
                            feature_data],axis=1)


In [44]:
#Loading trial logs of models
models= pd.read_csv("trial_logs.csv")
models.head()

,Model_Name,type,feat_select,Target_Dist,Data_Type,pozitif_acc_treshold_0.5,negatif_acc_treshold_0.5,pozitif_acc_treshold_0.7,negatif_acc_treshold_0.7,pozitif_acc_treshold_0.8,negatif_acc_treshold_0.8,pozitif_acc_treshold_0.9,negatif_acc_treshold_0.9
0,Log_Reg,Single,all_columns,binary,Train,0.559381,0.555171,0.718543,0.719298,1.0,1.0,NaN,NaN
1,Log_Reg,Single,all_columns,binary,Test,0.562443,0.557327,0.600000,0.888889,NaN,NaN,NaN,NaN
2,Log_Reg,Single,manuel_selected,binary,Train,0.553450,0.549571,0.691176,0.555556,1.0,NaN,NaN,NaN
3,Log_Reg,Single,manuel_selected,binary,Test,0.556289,0.552885,0.739130,1.000000,NaN,NaN,NaN,NaN
4,Log_Reg,Single,test_selected,binary,Train,0.559361,0.554888,0.719858,0.727273,1.0,NaN,NaN,NaN


In [45]:
#Selecting models test performance logs.
model_test=models[models.Data_Type=="Test"]
model_test.head()

,Model_Name,type,feat_select,Target_Dist,Data_Type,pozitif_acc_treshold_0.5,negatif_acc_treshold_0.5,pozitif_acc_treshold_0.7,negatif_acc_treshold_0.7,pozitif_acc_treshold_0.8,negatif_acc_treshold_0.8,pozitif_acc_treshold_0.9,negatif_acc_treshold_0.9
1,Log_Reg,Single,all_columns,binary,Test,0.562443,0.557327,0.600000,0.888889,NaN,NaN,NaN,NaN
3,Log_Reg,Single,manuel_selected,binary,Test,0.556289,0.552885,0.739130,1.000000,NaN,NaN,NaN,NaN
5,Log_Reg,Single,test_selected,binary,Test,0.563218,0.558551,0.658537,0.900000,NaN,NaN,NaN,NaN
7,Decistion_Tree,Single,all_columns,binary,Test,0.548235,0.547159,0.583333,0.406250,0.583333,0.666667,NaN,0.666667
9,Decistion_Tree,Single,manuel_selected,binary,Test,0.550302,0.538481,0.709677,0.600000,0.625000,NaN,0.625,NaN


In [46]:
#In order to select matches with most likely to be end over 2.5 goals
    # we will choose models where positive recall rate over 0.7 at given positive treshold
over_fixtures=[]

#In order to select matches with most likely to be end bellow 2.5 goals
    # we will choose models where negative recall rate over 0.7 at given negative treshold
below_fixtures=[]

#Treshold values for positive treshold (and 1-treshold for negatives)
tresholds=[0.7,0.8]
modes= ["pozitif","negatif"]

#Iterated over treshold list
for tresh in tresholds:
    
    #Itarated over modes list
    for mode in modes:
        
        #Determines trail log column 
        kolon="{}_acc_treshold_{}".format(mode,tresh)
        print(kolon)
        
        #If same model has more than one over 0.7 recall at given trail column
            #The record with most recall rate selected.
        selected_models=model_test[model_test[kolon]>=0.7].sort_values([kolon,"feat_select"]
                                                                       ,ascending=False).drop_duplicates("Model_Name").reset_index(drop=True)

        #Selected model count
        model_num=selected_models.shape[0]
        
        #Iterating over each selected models
        for num in range(model_num):
            
            #Selected model name
            model_names=selected_models["Model_Name"][num]
            
            #Selected with most succesfull feature set.
            feat_type=selected_models["feat_select"][num]
            
            
            if selected_models["type"][num]=="Single":
                model_names=[model_names]
            
            #If model is ensemble, each model in ensemble saved into a list
            else:
                print(model_names)
                listed=model_names[2:-2].split("'")
                model_names=[i for i in listed if i!=", "]
            
            
            prediction=np.array(0)
            #Iterated over each model in model_names list
            for model in model_names:
                print(model,feat_type)
                
                #Selected model dictonary loaded
                with open("{}/{}_{}".format(direc,model,feat_type),"rb") as mod:

                    saved_model= pickle.load(mod)
                #Model with trained weights selected
                model_obj=saved_model["Model_Obj"]
                
                #Possion and binary models treated differently
                if model!="XG_Poisson":
                    prediction=prediction+model_obj.predict_proba(incoming_matches_v1[saved_model["feats"]])[:,1]
                else:
                    
                    #If model prediction is count data, then it is normalized.
                    pred_xg=model_obj.predict(incoming_matches_v1[saved_model["feats"]])
                    pred_xg=pd.Series(pred_xg).apply(lambda x: 5 if x>5 else x)
                    prediction=prediction+pred_xg/5
                    
            #Sum of model predictions dived by model count
            prediction_final=prediction/len(model_names)
            #Given fixture plays matched with their predictions
            results=pd.DataFrame({"Preds":prediction_final,"fixture.id":incoming_matches_v1["fixture.id"]})
            
            #Select matches with bellow 1-treshold
            if mode=="negatif":
                below_fixtures=below_fixtures+list(results["fixture.id"][results.Preds<1-tresh].unique())
            
            #Select matches with above treshold.
            else:
                over_fixtures=over_fixtures+list(results["fixture.id"][results.Preds>tresh].unique())
            

pozitif_acc_treshold_0.7
['Log_Reg', 'Decistion_Tree', 'Random_Forrest']
Log_Reg test_selected
Decistion_Tree test_selected
Random_Forrest test_selected
['Log_Reg', 'Decistion_Tree']
Log_Reg all_columns
Decistion_Tree all_columns
['Decistion_Tree', 'Random_Forrest']
Decistion_Tree all_columns
Random_Forrest all_columns
Random_Forrest test_selected
Log_Reg manuel_selected
['Log_Reg', 'Random_Forrest']
Log_Reg test_selected
Random_Forrest test_selected
Decistion_Tree manuel_selected
negatif_acc_treshold_0.7
XG_Poisson test_selected
['Log_Reg', 'Random_Forrest']
Log_Reg test_selected
Random_Forrest test_selected
Log_Reg manuel_selected
['Log_Reg', 'Decistion_Tree']
Log_Reg all_columns
Decistion_Tree all_columns
['Decistion_Tree', 'Random_Forrest']
Decistion_Tree all_columns
Random_Forrest all_columns
['Log_Reg', 'Decistion_Tree', 'Random_Forrest']
Log_Reg all_columns
Decistion_Tree all_columns
Random_Forrest all_columns
Random_Forrest test_selected
pozitif_acc_treshold_0.8
['Log_Reg', 'De

In [47]:
below_fixtures

[]

In [48]:
over_fixtures

[1035419, 1048767, 1074416, 1035419]

In [49]:
#Selected fixture whoose prediction is over given treshold, selected from main data
over_matches=incoming_matches[incoming_matches["fixture.id"].isin(over_fixtures)][["league.country","league.name","teams.home.name","teams.away.name","fixture_date"]]
over_matches

,league.country,league.name,teams.home.name,teams.away.name,fixture_date
5,England,Premier League,Manchester City,Chelsea,2024-02-17
77,Germany,2. Bundesliga,SC Paderborn 07,Holstein Kiel,2024-02-17
219,Saudi-Arabia,Pro League,Al-Hilal Saudi FC,Al-Raed,2024-02-18


In [50]:
#Selected fixture whoose prediction is bellow given treshold, selected from main data
under_matches=incoming_matches[incoming_matches["fixture.id"].isin(below_fixtures)][["league.country","league.name","teams.home.name","teams.away.name","fixture_date"]]
under_matches

,league.country,league.name,teams.home.name,teams.away.name,fixture_date
